## Importing Necessary libraries

In [ ]:
!pip install langid

In [ ]:
import nltk
nltk.download('all')

In [ ]:
import pandas as pd 
import sklearn
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.datasets import fetch_20newsgroups

In [11]:
from textblob import TextBlob
import langid
langid.classify("This is a test")[0]


'en'

In [3]:
!unzip /content/drive/MyDrive/Textmining/tweets-analysis-main.zip

Archive:  /content/drive/MyDrive/Textmining/tweets-analysis-main.zip
740485a04f3f0e7de4d1f86fcb86c96604a3b2f4
   creating: tweets-analysis-main/
  inflating: tweets-analysis-main/Culture-tweets.csv  
  inflating: tweets-analysis-main/Economy-tweets.csv  
  inflating: tweets-analysis-main/Health-tweets.csv  
  inflating: tweets-analysis-main/Politics-tweets.csv  
  inflating: tweets-analysis-main/README.md  
  inflating: tweets-analysis-main/Social-tweets.csv  
  inflating: tweets-analysis-main/Sport-tweets.csv  
  inflating: tweets-analysis-main/analysis.ipynb  
  inflating: tweets-analysis-main/concatenation_datasets.ipynb  
  inflating: tweets-analysis-main/culture.ipynb  
  inflating: tweets-analysis-main/data.csv  
  inflating: tweets-analysis-main/economy.ipynb  
  inflating: tweets-analysis-main/environment.yml  
  inflating: tweets-analysis-main/health.ipynb  
  inflating: tweets-analysis-main/politics.ipynb  
  inflating: tweets-analysis-main/social.ipynb  
  inflating: tweets-

# Load&Unzip data

In [74]:
df_Culture = pd.read_csv("/content/tweets-analysis-main/Culture-tweets.csv", error_bad_lines=False)
df_Culture['category'] = [0] * len (df_Culture)
df_Health = pd.read_csv("/content/tweets-analysis-main/Economy-tweets.csv", error_bad_lines=False)
df_Health['category'] = [1] * len (df_Health)
df_Sports = pd.read_csv("/content/tweets-analysis-main/Sport-tweets.csv", error_bad_lines=False)
df_Sports['category'] = [2] * len (df_Health)
df_Politics = pd.read_csv("/content/tweets-analysis-main/Politics-tweets.csv", error_bad_lines=False)
df_Politics['category'] = [3] * len (df_Politics)


In [75]:
dataset = pd.concat([df_Culture, df_Health, df_Sports, df_Politics],ignore_index=True)
dataset.head()

,user,Text,category
0,_sha_kyra,RT @Black_Archange: Aujourd’hui en France : \n...,0
1,penaraimakmaiei,RT @NctzenToday: จริงๆซือจือนี่ไทยก็โดนกันมาหล...,0
2,J_free_98,RT @okayplayer: Breakdancing has officially be...,0
3,_skywalker8,RT @shroomimimi: you know who felt uncomfortab...,0
4,Rudeboi13_,"@King_Seb13 It’s so true, our culture is all a...",0


In [76]:
dataset.to_csv( "data.csv", index=False, encoding='utf-8-sig')


In [77]:
df = dataset.sample(frac=1)


In [78]:
df.head()

,user,Text,category
10951,maria82598353,RT @myhlee: NEW: Trump asks Pennsylvania House...,3
5691,W98556374,RT @OSSRobertSteele: The Steele Report – Grade...,1
221,SagebrushTimes,Governors of Western USA burning culture out o...,0
976,DelatourRegis,@samsedz @to_t0bi @CancerReligieux @uwukoyo 😂😂...,0
2740,itzeldiaz29,RT @AnnKumfer: ☄️High risk conditions (such as...,0


In [79]:
len(df)

14000

In [80]:
(df['category'].value_counts())

3    5000
2    3000
1    3000
0    3000
Name: category, dtype: int64

In [81]:
df.shape
df = df.drop(['user'],axis= 1)

In [82]:
df.shape

(14000, 2)

# Part 1: Category classification

## Data Preprocessing

In [83]:
import pandas as pd
import numpy as np
import re
# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
# Gensim
import gensim
from gensim.utils import simple_preprocess
# NLTK
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize


In [84]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [85]:
from nltk.stem import WordNetLemmatizer  # NLTK LEMMATIZER
from nltk.stem.porter import PorterStemmer  # NLTK STEMMER

def lemmatize_stemming(text,lemmatize=True):
    ps = PorterStemmer()
    lem = WordNetLemmatizer().lemmatize(text)
    stem = ps.stem(lem)
    if lemmatize:
      return lem
    return stem


In [86]:
>>> from nltk.tokenize import TweetTokenizer
>>> tknzr = TweetTokenizer()


In [87]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = tknzr.tokenize(text)
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    # Removing multiple spaces
    text = re.sub(r'\s+', ' ', text)
    #text = re.sub("[rt]"," ",text)
    # remove stop words
    if remove_stopwords:
        text = tknzr.tokenize(text)
        stops = set(stopwords.words("english"))
        text = [ (w) for w in text if not w in stops and len(w)>3]
        text = " ".join(text)

    # Tokenize each word
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return ' '.join(text)

In [88]:
import numpy as np
df['Text'] = df['Text'].map(lambda x: x if langid.classify(x)[0] == 'en' else np.nan)

In [89]:
len(df)

14000

In [90]:
df = df.dropna(axis=0)

In [91]:
len(df)

11768

In [92]:
df['preprocessed_text'] = df['Text'].map(lambda x: clean_text(x))

In [93]:
df.head()

,Text,category,preprocessed_text
10951,RT @myhlee: NEW: Trump asks Pennsylvania House...,3,myhlee trump asks pennsylvania house speaker h...
5691,RT @OSSRobertSteele: The Steele Report – Grade...,1,ossrobertsteele steele report grades topics to...
221,Governors of Western USA burning culture out o...,0,governors western burning culture communities
2740,RT @AnnKumfer: ☄️High risk conditions (such as...,0,annkumfer high risk conditions meningitis endo...
9768,RT @a_sandhan: Middle obsession with corruptio...,3,sandhan middle obsession corruption issues cle...


## Splitting dataset into train/test set


In [94]:
>>> X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df['preprocessed_text'], df['category'], test_size=0.2, random_state=42)

## Modelling

### SVM

In [95]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

SVM_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

SVM_clf.fit(X_train, y_train)


predicted = SVM_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))


              precision    recall  f1-score   support

           0       0.92      0.87      0.89       521
           1       0.86      0.87      0.87       525
           2       0.91      0.89      0.90       410
           3       0.89      0.93      0.91       898

    accuracy                           0.90      2354
   macro avg       0.90      0.89      0.89      2354
weighted avg       0.90      0.90      0.90      2354



### Random Forest

In [96]:
from sklearn.ensemble import RandomForestClassifier
RF_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=100)),
                     ])

RF_clf.fit(X_train, y_train)


predicted = RF_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))


              precision    recall  f1-score   support

           0       0.92      0.87      0.89       521
           1       0.88      0.83      0.85       525
           2       0.93      0.88      0.91       410
           3       0.86      0.94      0.89       898

    accuracy                           0.89      2354
   macro avg       0.90      0.88      0.89      2354
weighted avg       0.89      0.89      0.89      2354



### Naive Bayes

In [97]:
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))


              precision    recall  f1-score   support

           0       0.93      0.80      0.86       521
           1       0.83      0.82      0.82       525
           2       0.93      0.83      0.87       410
           3       0.82      0.93      0.87       898

    accuracy                           0.86      2354
   macro avg       0.88      0.84      0.86      2354
weighted avg       0.87      0.86      0.86      2354



### k-Nearest Neighbors



In [98]:
from sklearn.neighbors import KNeighborsClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', (KNeighborsClassifier())),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))


              precision    recall  f1-score   support

           0       0.72      0.53      0.61       521
           1       0.74      0.48      0.59       525
           2       0.62      0.33      0.43       410
           3       0.57      0.90      0.70       898

    accuracy                           0.62      2354
   macro avg       0.66      0.56      0.58      2354
weighted avg       0.65      0.62      0.60      2354



### Saving best model


In [99]:
import pickle
pickle.dump(SVM_clf, open("SVM_clf", 'wb'))

# Part 2: Twitter Sentiment Analysis

In [ ]:
# preparing sentiment analysis dataset
#df = df.drop(['Text', 'category'], axis=1)

In [64]:
df.head()

,user,preprocessed_text
10064,gunther_nask,eeberquist know trying introduce young readers...
5319,bondzie007,timmslaney pierrepoilievre many investors peop...
12,skiino8,coachckennedy time tulane great experience cul...
11739,KamalHo52194980,khushnumakashm army organised selfdefence clas...
10913,lance_vamplew,therealukecage super voters practice politics ...


*Calculating sentiment polarity for each tweet*

In [100]:
#generating polarity values
df['polarity']= df['preprocessed_text'].map(lambda x: TextBlob(x).sentiment.polarity)

In [69]:
df.head()

,user,preprocessed_text,polarity
10064,gunther_nask,eeberquist know trying introduce young readers...,0.025000
5319,bondzie007,timmslaney pierrepoilievre many investors peop...,0.250000
12,skiino8,coachckennedy time tulane great experience cul...,0.800000
11739,KamalHo52194980,khushnumakashm army organised selfdefence clas...,0.000000
10913,lance_vamplew,therealukecage super voters practice politics ...,0.166667


In [70]:
df['polarity'].value_counts()

 0.000000    5031
-0.500000     289
-0.050000     289
 0.200000     288
-0.250000     278
             ... 
-0.175000       1
 0.056250       1
-0.263889       1
-0.245833       1
 0.188889       1
Name: polarity, Length: 555, dtype: int64

In [101]:
df['sentiment'] = df['polarity'].map(lambda x: 1  if (x>0) else (-1 if x<0 else x))

In [102]:
df.head()

,Text,category,preprocessed_text,polarity,sentiment
10951,RT @myhlee: NEW: Trump asks Pennsylvania House...,3,myhlee trump asks pennsylvania house speaker h...,0.000000,0.0
5691,RT @OSSRobertSteele: The Steele Report – Grade...,1,ossrobertsteele steele report grades topics to...,-0.100000,-1.0
221,Governors of Western USA burning culture out o...,0,governors western burning culture communities,0.000000,0.0
2740,RT @AnnKumfer: ☄️High risk conditions (such as...,0,annkumfer high risk conditions meningitis endo...,0.160000,1.0
9768,RT @a_sandhan: Middle obsession with corruptio...,3,sandhan middle obsession corruption issues cle...,0.183333,1.0


In [103]:
df['sentiment'].value_counts()

 0.0    5031
 1.0    4092
-1.0    2645
Name: sentiment, dtype: int64

In [105]:
df.groupby(['category', 'sentiment'])['category','sentiment'].count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


category  sentiment
category sentiment                     
0        -1.0            563        563
          0.0           1124       1124
          1.0            818        818
1        -1.0            485        485
          0.0           1121       1121
          1.0           1049       1049
2        -1.0            317        317
          0.0            952        952
          1.0            822        822
3        -1.0           1280       1280
          0.0           1834       1834
          1.0           1403       1403

In [106]:
>>> X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df['preprocessed_text'], df['sentiment'], test_size=0.2, random_state=42)


In [107]:
SVM_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

SVM_clf.fit(X_train, y_train)

predicted = SVM_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

        -1.0       0.94      0.84      0.89       540
         0.0       0.86      0.96      0.91       963
         1.0       0.94      0.88      0.91       851

    accuracy                           0.90      2354
   macro avg       0.91      0.89      0.90      2354
weighted avg       0.91      0.90      0.90      2354



In [ ]:
#saving model 
pickle.dump(SVM_clf, open("SVM_sent_clf", 'wb'))

# Part 3: Topic Modelling

**Installing pyLDAvis for visualization**

In [108]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 12.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ab97d635a4b5bb0c1a273c14343117858d0e9d0597fcbc18ec6859d493a7eaee
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [109]:
from sklearn.decomposition import LatentDirichletAllocation
import logging
import pyLDAvis
import warnings


*Tokenizing tweets*

In [110]:
df['tweets_tokenized'] = df['preprocessed_text'].map(lambda x: tknzr.tokenize(x))

In [111]:
df.head()

,Text,category,preprocessed_text,polarity,sentiment,tweets_tokenized
10951,RT @myhlee: NEW: Trump asks Pennsylvania House...,3,myhlee trump asks pennsylvania house speaker h...,0.000000,0.0,"[myhlee, trump, asks, pennsylvania, house, spe..."
5691,RT @OSSRobertSteele: The Steele Report – Grade...,1,ossrobertsteele steele report grades topics to...,-0.100000,-1.0,"[ossrobertsteele, steele, report, grades, topi..."
221,Governors of Western USA burning culture out o...,0,governors western burning culture communities,0.000000,0.0,"[governors, western, burning, culture, communi..."
2740,RT @AnnKumfer: ☄️High risk conditions (such as...,0,annkumfer high risk conditions meningitis endo...,0.160000,1.0,"[annkumfer, high, risk, conditions, meningitis..."
9768,RT @a_sandhan: Middle obsession with corruptio...,3,sandhan middle obsession corruption issues cle...,0.183333,1.0,"[sandhan, middle, obsession, corruption, issue..."


*Vizualizing an example*

In [116]:
df['Text'][10951]

'RT @myhlee: NEW: Trump asks Pennsylvania House speaker for help overturning election results, personally intervening in a third state. Scoo…'

In [117]:
df['preprocessed_text'][10951]

'myhlee trump asks pennsylvania house speaker help overturning election results personally intervening third state scoo'

*Preparing our id2word dictionarry*

In [118]:
dictionary = gensim.corpora.Dictionary(df['tweets_tokenized'])

In [119]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 asks
1 election
2 help
3 house
4 intervening
5 myhlee
6 overturning
7 pennsylvania
8 personally
9 results
10 scoo


In [277]:
#dictionary.filter_extremes(no_below=5, keep_n=100000)

*Preparing BOW Corpus* // 

In [120]:
bow_corpus = [dictionary.doc2bow(doc) for doc in df['tweets_tokenized']]
bow_corpus[4310]

[(78, 1),
 (793, 1),
 (1962, 1),
 (5911, 1),
 (9255, 1),
 (9256, 1),
 (9257, 1),
 (9258, 1)]

*Training LDA model*

In [ ]:
num_topics = 4
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=num_topics, id2word=dictionary)

*Printing topics generated by LDA*

In [126]:
from pprint import pprint
# number of topics
pprint(lda_model.print_topics())


[(0,
  '0.020*"sport" + 0.015*"biden" + 0.014*"defense" + 0.014*"austin" + '
  '0.013*"culture" + 0.012*"lloyd" + 0.011*"secretary" + 0.011*"retired" + '
  '0.009*"stan" + 0.009*"going"'),
 (1,
  '0.028*"economy" + 0.013*"politics" + 0.010*"covid" + 0.010*"culture" + '
  '0.010*"people" + 0.008*"like" + 0.008*"would" + 0.008*"sport" + '
  '0.007*"trump" + 0.006*"best"'),
 (2,
  '0.020*"culture" + 0.011*"sport" + 0.007*"trump" + 0.006*"games" + '
  '0.005*"olympic" + 0.004*"economy" + 0.003*"politics" + 0.003*"people" + '
  '0.003*"president" + 0.003*"know"'),
 (3,
  '0.034*"trump" + 0.022*"election" + 0.021*"pennsylvania" + 0.019*"speaker" + '
  '0.018*"house" + 0.018*"help" + 0.016*"results" + 0.013*"asks" + '
  '0.012*"overturning" + 0.010*"state"')]


## LDA Topics Visualization

In [129]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/content/drive/MyDrive/Textmining/results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/content/drive/MyDrive/Textmining/results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.071186  0.157171       1        1  28.731049
3      0.202605 -0.000214       2        1  26.064376
0     -0.071999 -0.151469       3        1  23.906616
2     -0.059419 -0.005488       4        1  21.297960, topic_info=             Term        Freq        Total Category  logprob  loglift
7    pennsylvania  724.000000   724.000000  Default  30.0000  30.0000
2            help  580.000000   580.000000  Default  29.0000  29.0000
1        election  746.000000   746.000000  Default  28.0000  28.0000
11        speaker  655.000000   655.000000  Default  27.0000  27.0000
9         results  513.000000   513.000000  Default  26.0000  26.0000
..            ...         ...          ...      ...      ...      ...
771          stan   75.313323   355.696681   Topic4  -5.7961  -0.0059
150       economy  106.197793  1358.756330   Topic4  -5.4525  -1.0025
582          make   64.979330   222.850495   Topic4  -5.9437   0.3141
53       politics   85.544171   761.145357   Topic4  -5.6687  -0.6392
106        people   79.290997   615.566087   Topic4  -5.7446  -0.5029

[289 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
772       3  0.993451     ableist
772       4  0.005844     ableist
384       1  0.973849     actions
384       3  0.017869     actions
766       1  0.031260  activities
...     ...       ...         ...
921       4  0.420886        year
604       1  0.020999       youth
604       2  0.062998       youth
604       3  0.083998       youth
604       4  0.839980       youth

[616 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

## Testing model on unseen document

In [131]:
unseen_document = 'Captain , Leader and future legend masonmount_10 great performance, onto the next round Ok handBlue heart CFC myteam '
bow_vector = dictionary.doc2bow([clean_text(unseen_document)])

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 4)))

Score: 0.25	 Topic: 0.020*"sport" + 0.015*"biden" + 0.014*"defense" + 0.014*"austin"
Score: 0.25	 Topic: 0.028*"economy" + 0.013*"politics" + 0.010*"covid" + 0.010*"culture"
Score: 0.25	 Topic: 0.020*"culture" + 0.011*"sport" + 0.007*"trump" + 0.006*"games"
Score: 0.25	 Topic: 0.034*"trump" + 0.022*"election" + 0.021*"pennsylvania" + 0.019*"speaker"
